In [1]:
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.drift_detection.adwin import ADWIN

from skmultiflow.data import FileStream             # create stream from file
from prequential import run_prequential

In [2]:
data_loader = FileStream(filepath='../out.csv')

In [3]:
ref_sample, _ = data_loader.next_sample(50)
data_loader.reset()

In [ ]:
best_params = None
best_score = 0
best_metrics = []
results = []
data_loader.restart()

accuracy, precision, recall, f1, avg_processing_time = run_prequential(
    classifier=AdaptiveRandomForestClassifier(
        drift_detection_method=ADWIN(0.0005)
    ),
    stream=data_loader,
    drift_detector=ADWIN(delta=0.0005),
    preq_samples=100000
)

print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}")
print(f"Average processing time: {avg_processing_time}")

Accuracy: 0.999410, Precision: 0.999722, Recall: 0.999314, F1: 0.999518
Average processing time: 0.003885102801999991
